In [23]:
import Constants
from  openai import OpenAI
import os
import numpy as np
import pandas as pd
import time

import gspread
import gspread_dataframe as gd
import gspread_formatting as gf
from gspread_formatting import cellFormat, color, textFormat


## Quiz Module

In [ ]:
from typing import List, Dict, Any, Tuple, Union

import Constants
from  openai import OpenAI
import os
import numpy as np
import pandas as pd
import time

import os
from io import StringIO
from datetime import datetime

from main.gsheets import load_dict, save_df_to_gsheet
from main.translation import save_new_words_to_dict

client = OpenAI(
    api_key = Constants.API_KEY_OPENAI,
)

def get_completion(prompt, model="gpt-4o-mini", temperature=0):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    return response


def get_prompt_generate_word_quiz(
    word_dict: pd.DataFrame,
    startfrom_date_filter: str = None,
    category_filter: str = None
    ) -> str:
        
    if startfrom_date_filter:
        word_dict = word_dict.loc[word_dict['Added Date'] >= startfrom_date_filter]

    if category_filter:
            word_dict = word_dict.loc[word_dict['Word Category'] == category_filter]

    word_list = word_dict['Word'].drop_duplicates().sample(10).values
    prompt = f''' 
    Given this word list:
    {word_list}

    Can you create an Mandarin exercise where you choose 10 non-duplicated words and an example sentence using them.   
    Leaving 2 blank columns where user can input the pinyin and the meaning in English.

    The output should only be a 10x4 table with no other written text.  The table should have the following columns:
    1) Word
    2) Sentence
    3) Pinyin (Leave blank)
    4) Meaning (Leave blank)
    '''

    return prompt

# 4O Maybe Necessary to Evaluate Quiz.  Mini seems to be halluciating quite a bit. 
def get_prompt_evaluate_quiz_meaning_only(
        word_list, 
        sentence_list, 
        meaning
    ) -> str:
    prompt = f'''
    Please act as Chinese teacher and evaluate the following quiz.  The student was asked to provide the meaning for the following words.
    The context for the word is provided in the sentence.  Please check whether the meaning provided is correct.  If not, please provide the correct meaning.
    Each word should be evaluated separately.
    
    Word List: {word_list}
    Provided Sentence: {sentence_list}
    
    Please generate the following table with the following columns: 
    Word List (Column #1):  The word list provided
    Meaning (Column #2):  The meaning provided by the students {meaning}
    Meaning Correct (Column #3):  Check whether the Provided Meaning is correct given the context sentence.  The meaning of the word has to evaluated in the context of the sentence provided. Give just yes/no response.  
    Meaning Correction (Column #4):  If column #3 is "no" provide short explanation of why the meaning is incorrect and provide correct answer.  The explanation should not be longer than 20 words.  If column #6 is "yes" then must be blank.  

    The table columns must be (1) Word List, (2) Meaning, (3) Meaning Correct, (4) Meaning Correction
    No changes to the column name is allowed.
    No other response should be given except the table
    '''

    return prompt

# 4O Maybe Necessary to Evaluate Quiz.  Mini seems to be halluciating quite a bit. 
def get_prompt_evaluate_quiz(
        word_list, 
        sentence_list, 
        pinyin, 
        meaning
    ) -> str:
    prompt = f'''
    Here's the answer provided.  Please check them for correctness and mark any incorrect answer and provide the correct one. 

    Word List: {word_list}
    Provided Sentence: {sentence_list}
    
    Please generate the following table with the following columns: 
    Word List (Column #1):  The word list provided
    Pinyin (Column #2): Should be a value from this list {pinyin}
    Meaning (Column #3):  Should be a value from this list {meaning}
    Pinyin Correct (Column #4):  Check whether the Provided Pinyin is correct relative to the Word List.  Give just yes/no response.
    Correct Pinyin (Column #5):  Should contain the correct pinyin if the answer is incorrect.  Blank otherwise. When pinyin is incorrect only the correct pinyin should be provided, no other text is allowed
    Meaning Correct (Column #6):  Check whether the Provided Meaning is correct relative to the Word List.  Give just yes/no response.  
    Correct Meaning (Column #7):  If column #6 is "no" provide short explanation of why the meaning is incorrect and provide correct answer.  If column #6 is "yes" then must be blank. 

    The tone for the pinyin will be provided with number 1, 2, 3, 4, 5
    The special character ü⁠ can be replaced by v in the answer

    No other response should be given except the table
    '''
    return prompt


def parse_meaning_table(content: str) -> pd.DataFrame:
    '''
    Parse the table response from OpenAI into a pandas DataFrame
    '''
    data = StringIO(content)

    # Read the table into a pandas DataFrame
    df = pd.read_csv(data, delimiter='|',  engine='python')
    df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
    col_to_keep = [col for col in df if 'Unnamed' not in col]
    df = df[col_to_keep]
    df.columns = df.columns.str.strip()

    col = df.columns[0]
    df = df.loc[~df[col].str.contains('--')]
    return df


class QuizGenerator:
    def __init__(
            self, 
            gsheet_name: str,
            wks_name: str
            ):
        self.gsheet_name = gsheet_name
        self.wks_name = wks_name
        self.dict_df = load_dict(gsheet_mode=True, gsheet_name=gsheet_name, worksheet_name=wks_name)

    def generate_pinyin_and_meaning_quiz(
            self, 
            id_column: str = 'Word Id',
            num_words: int = 10,
            date_filter: Union[List[str], str] = None,
            category_filter: Union[List[str], str] = None, 
            rarity_filter: Union[List[str], str] = None
        ) -> pd.DataFrame:
        dict_df = self.dict_df.copy()

        if date_filter is not None:
            dict_df = dict_df[dict_df['Added Date'] >= date_filter]

        if category_filter is not None:
            if type(category_filter) == str:
                category_filter = [category_filter]
                dict_df = dict_df[dict_df['Word Category'].isin(category_filter)]

        if rarity_filter is not None:
            if type(rarity_filter) == str:
                rarity_filter = [rarity_filter]
                dict_df = dict_df[dict_df['Word Rarity'].isin(rarity_filter)]

        unique_ids = dict_df[id_column].unique()
        num_to_select = min(num_words, len(unique_ids))
        quiz_id = pd.Series(unique_ids).sample(n=num_to_select)  

        quiz_df = dict_df.loc[dict_df[id_column].isin(quiz_id)]
        quiz_answer_key = quiz_df[['Word Id', 'Word', 'Sentence', 'Pinyin', 'Pinyin Simplified']].sample(frac=1, random_state=1).reset_index(drop=True)
        quiz_df = quiz_answer_key.drop(columns=['Pinyin', 'Pinyin Simplified'])
        quiz_df['Pinyin'] = ''
        quiz_df['Meaning'] = '' 
        
        self.answer_key = quiz_answer_key
        self.quiz = quiz_df
        return quiz_df


    def check_pinyin(
            self,
            pinyin_answer
        ):
        '''
        This function checks the pinyin for the quiz provided.
        '''
        answer_key_df = self.answer_key
        quiz_df = self.quiz

        pinyin_eval_df = answer_key_df.loc[answer_key_df['Word Id'].isin(quiz_df['Word Id'])][['Word Id', 'Word', 'Pinyin', 'Pinyin Simplified']].reset_index (drop=True)
        pinyin_eval_df['Pinyin Answer'] = pinyin_answer
        pinyin_eval_df['Pinyin Simplified'] = pinyin_eval_df['Pinyin Simplified'].apply(lambda x: x.replace(' ', '').lower().replace('5', ''))
        pinyin_eval_df['Pinyin Answer'] = pinyin_eval_df['Pinyin Answer'].apply(lambda x: x.replace(' ', '').lower().replace('5', ''))
        pinyin_eval_df['Pinyin Correct'] = (pinyin_eval_df['Pinyin Simplified'] == pinyin_eval_df['Pinyin Answer'])
        pinyin_eval_df['Pinyin Correction'] = np.where(pinyin_eval_df['Pinyin Correct'], '', pinyin_eval_df['Pinyin'])
        pinyin_eval_df['Pinyin Correct'] = pinyin_eval_df['Pinyin Correct'].map({True: 'yes', False: 'no'})

        self.pinyin_eval_df = pinyin_eval_df
        return pinyin_eval_df

    def check_meaning(
            self,
            meaning_answer
        ):
        '''
        This function checks the meaning for the quiz provided.
        '''
        quiz_df = self.quiz
        quiz_prompt = get_prompt_evaluate_quiz_meaning_only(
            word_list = quiz_df['Word'].values,
            sentence_list = quiz_df['Sentence'].values,
            meaning = meaning_answer
            )

        sample_response_translation = get_completion(prompt=quiz_prompt, temperature=0.7)
        meaning_eval_df = parse_meaning_table(sample_response_translation.choices[0].message.content)
        meaning_eval_df = meaning_eval_df.reset_index(drop=True)

        self.meaning_eval_df = meaning_eval_df
        return meaning_eval_df


    def evaluate_pinyin_and_meaning_quiz(
            self,
            pinyin_answer,
            meaning_answer
        ):
        pinyin_eval_df = self.check_pinyin(
            pinyin_answer = pinyin_answer
            )
        meaning_eval_df = self.check_meaning(
            meaning_answer = meaning_answer
            )

        outdf = pd.concat([
            self.quiz[['Word', 'Sentence']], 
            pinyin_eval_df[['Pinyin Answer', 'Pinyin Correct', 'Pinyin Correction']], 
            meaning_eval_df[['Meaning', 'Meaning Correct', 'Meaning Correction']]
            ], axis=1).set_index(pinyin_eval_df['Word Id'])
        
        self.quiz_result = outdf
        return outdf

    def update_quiz_score(
            self,
            gsheet_name: str,
            wks_name: str
        ) -> None:
        '''
        This function updates the quiz score for the word dictionary.
        '''
        quiz_result_df = self.quiz_result
        word_dict = load_dict(gsheet_mode=True, gsheet_name=gsheet_name, worksheet_name=wks_name)

        quiz_result_df['Right Score'] = np.where((quiz_result_df['Meaning Correct']=="yes")&(quiz_result_df['Pinyin Correct']=="yes"), 1, 0)
        quiz_result_df['Wrong Score'] = np.where(quiz_result_df['Right Score']==0, 1, 0)
        quiz_result_df['Last Quiz'] = datetime.now().strftime('%Y-%m-%d')

        word_dict_quiz_export = word_dict.merge(quiz_result_df[['Right Score', 'Wrong Score', 'Last Quiz']].reset_index(), on='Word Id', how='left')

        num_attempts = word_dict_quiz_export['Num_Quiz_Attempt'].fillna(0).astype(int).sum()
        word_dict_quiz_export['Num_Quiz_Attempt'] = word_dict_quiz_export['Num_Quiz_Attempt'].astype(int) + word_dict_quiz_export['Right Score'].notna()
        word_dict_quiz_export['Num_Correct'] = word_dict_quiz_export['Num_Correct'].astype(int) + word_dict_quiz_export['Right Score'].fillna(0)
        word_dict_quiz_export['Num_Wrong'] = word_dict_quiz_export['Num_Wrong'].astype(int) + word_dict_quiz_export['Wrong Score'].fillna(0)
        word_dict_quiz_export['Last_Quiz'] = word_dict_quiz_export['Last_Quiz'].fillna('2001-01-01')
        word_dict_quiz_export['Last_Quiz'] = np.where(word_dict_quiz_export['Last_Quiz'] < word_dict_quiz_export['Last Quiz'], word_dict_quiz_export['Last Quiz'], word_dict_quiz_export['Last_Quiz'])

        num_attempts_post_update = word_dict_quiz_export['Num_Quiz_Attempt'].fillna(0).astype(int).sum()
        num_correct_post_update = word_dict_quiz_export['Num_Correct'].fillna(0).astype(int).sum()
        num_wrong_post_update = word_dict_quiz_export['Num_Wrong'].fillna(0).astype(int).sum()

        attempt_count_condition = (num_attempts + len(quiz_result_df) != num_attempts_post_update)
        correct_wrong_total_condition = (num_correct_post_update + num_wrong_post_update != num_attempts_post_update)

        if (attempt_count_condition) or (correct_wrong_total_condition):
            raise Exception("Error in updating the quiz results.  Please check the code.")

        word_dict_quiz_export = word_dict_quiz_export.drop(columns=['Right Score', 'Wrong Score', 'Last Quiz'])

        save_df_to_gsheet(overwrite_mode=True, df_to_save=word_dict_quiz_export, gsheet_name=gsheet_name, wks_name=wks_name)

        return "Quiz Result Updated"

## Generating Word Quiz

In [2]:
from main.quiz import *

temp = 0.7
dict_sheet_name = "Tua_List"
gsheet_name = "New Chinese Words"

In [3]:
word_dict = load_dict(gsheet_mode=True, gsheet_name=gsheet_name, worksheet_name=dict_sheet_name)
word_dict.head(2)

,Word Id,Word,Pinyin,Pinyin Simplified,Type,Word Category,Word Rarity,Meaning,Sentence,Sentence Pinyin,Sentence Meaning,Added Date,Num_Quiz_Attempt,Num_Correct,Num_Wrong,Last_Quiz
1,1,帮助,bang1 zhu4,bang1 zhu4,Noun/Verb,Support,Common,Help/Assistance,我可以帮助你学习中文,Wǒ kěyǐ bāngzhù nǐ xuéxí Zhōngwén.,I can help you study Chinese.,2024-09-21,0,0,0,
2,2,包裹,bao1 guo3,bao1 guo3,Noun,Object,Common,Package,他昨天收到了一个包裹。,Wǒ zuótiān shōudào le bāoguǒ.,I received the package yesterday.,2024-09-21,0,0,0,


In [48]:
quiz_generator = QuizGenerator(word_dict)

In [49]:
quiz_df = quiz_generator.generate_pinyin_and_meaning_quiz(
        num_words = 10, 
        date_filter = '2024-10-10', 
        #category_filter = 'Fashion',
        rarity_filter = 'Common'
        )

quiz_df_display = quiz_df.drop(columns=['Word Id'])
quiz_df_display

,Word,Sentence,Pinyin,Meaning
0,延迟,会议延迟了一个小时。,,
1,留下,他留下了一封信。,,
2,点燃,他用火柴点燃了篝火。,,
3,意见,他对这个计划有不同的意见。,,
4,流,河水流得很快。,,
5,剩,这道菜剩下了很多。,,
6,无人,这个地方无人问津。,,
7,换,我需要换一下我的计划。,,
8,购买,我计划下周去商店购买一些新衣服。,,
9,度过,我们一起度过了一个快乐的假期。,,


In [28]:
quiz_df, answer_key = generate_pinyin_and_meaning_quiz(
    word_dict, 
    id_column='Word Id', 
    random_params={
        'num_words': 10, 
        'date_filter': '2024-10-10', 
        #'category_filter': 'Fashion'
        'rarity_filter': 'Common'
        }
    )

quiz_df_display = quiz_df.drop(columns=['Word Id'])
quiz_df_display

,Word,Sentence
0,愿意,我愿意帮助你。
1,接近,我们的目标是接近客户的需求。
2,文章,我在杂志上读到了一篇有趣的文章。
3,提,请提你的意见。
4,保护,我们需要保护环境。
5,住在,我住在北京。
6,烦恼,他总是为工作感到烦恼。
7,麻烦,麻烦你帮我一下。
8,白色,她穿了一件白色的裙子。
9,那天,你还记得那天我们一起去看电影吗？


In [50]:
pinyin_answer = 'an1jing4, bie2, umm, ded, fan4, jiu2, yu4jian4, jiao3, you1mo4, sheng1bing4'.split(', ')
meaning_answer = 'quiet, dont, ded, to spin, to make a mistake, old, to meet, feet, humor, sick'.split(', ')

In [51]:
quiz_df_display['Pinyin'] = pinyin_answer
quiz_df_display['Meaning'] = meaning_answer

In [52]:
quiz_result = quiz_generator.evaluate_pinyin_and_meaning_quiz(
    pinyin_answer = quiz_df_display['Pinyin'], 
    meaning_answer = quiz_df_display['Meaning']
)

quiz_result

C:\Users\wongs\AppData\Local\Temp\ipykernel_7276\1553284521.py:125: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


,Word,Sentence,Pinyin Answer,Pinyin Correct,Pinyin Correction,Meaning,Meaning Correct,Meaning Correction
Word Id,,,,,,,,
808,延迟,会议延迟了一个小时。,an1jing4,no,yán chí,quiet,no,'quiet' is incorrect; it means 'to delay'.
1508,留下,他留下了一封信。,bie2,no,liú xià,dont,no,'dont' is incorrect; it means 'to leave behind'.
1138,点燃,他用火柴点燃了篝火。,umm,no,diǎn rán,ded,no,'ded' is incorrect; it means 'to ignite'.
845,意见,他对这个计划有不同的意见。,ded,no,yì jiàn,to spin,no,'to spin' is incorrect; it means 'opinion'.
315,流,河水流得很快。,fan4,no,liú,to make a mistake,no,'to make a mistake' is incorrect; it means 'to...
821,剩,这道菜剩下了很多。,jiu2,no,shèng,old,no,'old' is incorrect; it means 'to remain'.
567,无人,这个地方无人问津。,yu4jian4,no,wú rén,to meet,no,'to meet' is incorrect; it means 'no one'.
1239,换,我需要换一下我的计划。,jiao3,no,huàn,feet,no,'feet' is incorrect; it means 'to exchange'.
1369,购买,我计划下周去商店购买一些新衣服。,you1mo4,no,gòu mǎi,humor,no,'humor' is incorrect; it means 'to buy'.


In [ ]:
quiz_generator.update_quiz_score(
    gsheet_name = gsheet_name, 
    wks_name = 'Tua_List'
    )

In [22]:
quiz_result = evaluate_pinyin_and_meaning_quiz(
    answer_key_df = answer_key, 
    quiz_df = quiz_df, 
    pinyin_answer = quiz_df['Pinyin'], 
    meaning_answer = quiz_df['Meaning']
)

update_quiz_score(
    quiz_result_df = quiz_result, 
    gsheet_name = gsheet_name, 
    wks_name = 'Tua_List'
    )

quiz_result

c:\Users\wongs\Documents\DS Work\repo\Pytorch LLM\GPT4LanguageLearning\main\quiz.py:125: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


,Word,Sentence,Pinyin Answer,Pinyin Correct,Pinyin Correction,Meaning,Meaning Correct,Meaning Correction,Right Score,Wrong Score,Last Quiz
Word Id,,,,,,,,,,,
410,赶走,我们需要赶走那些害虫。,an1jing4,no,gǎn zǒu,quiet,no,"Incorrect meaning; it means ""to drive away.""",0,1,2024-12-15
1461,询问,他询问了老师今天的作业内容。,bie2,no,xún wèn,dont,no,"Incorrect meaning; it means ""to inquire.""",0,1,2024-12-15
980,有趣,这部电影很有趣。,umm,no,yǒuqù,ded,no,"Incorrect meaning; it means ""interesting.""",0,1,2024-12-15
887,发生,事故发生在凌晨。,ded,no,fā shēng,to spin,no,"Incorrect meaning; it means ""to happen.""",0,1,2024-12-15
350,记得,我记得他的生日。,fan4,no,jì de,to make a mistake,no,"Incorrect meaning; it means ""to remember.""",0,1,2024-12-15
869,连,他连饭都没吃就走了。,jiu2,no,lián,old,no,"Incorrect meaning; it means ""even"" or ""to link.""",0,1,2024-12-15
399,距离,这段距离让我感到不安。,yu4jian4,no,jùlí,to meet,no,"Incorrect meaning; it means ""distance.""",0,1,2024-12-15
1343,偏,这条路偏离了原来的方向。,jiao3,no,piān,feet,no,"Incorrect meaning; it means ""to deviate.""",0,1,2024-12-15
1408,适合,这件衣服很适合你。,you1mo4,no,shì hé,humor,no,"Incorrect meaning; it means ""to fit.""",0,1,2024-12-15


##### Check Update Condition

In [ ]:
quiz_result_df = quiz_result, 
gsheet_name = gsheet_name, 
wks_name = 'Tua_List'

word_dict = load_dict(gsheet_mode=True, gsheet_name=gsheet_name, worksheet_name=wks_name)

quiz_result_df['Right Score'] = np.where((quiz_result_df['Meaning Correct']=="yes")&(quiz_result_df['Pinyin Correct']=="yes"), 1, 0)
quiz_result_df['Wrong Score'] = np.where(quiz_result_df['Right Score']==0, 1, 0)
quiz_result_df['Last Quiz'] = datetime.now().strftime('%Y-%m-%d')

word_dict_quiz_export = word_dict.merge(quiz_result_df[['Right Score', 'Wrong Score', 'Last Quiz']].reset_index(), on='Word Id', how='left')

num_attempts = word_dict_quiz_export['Num_Quiz_Attempt'].fillna(0).astype(int).sum()
word_dict_quiz_export['Num_Quiz_Attempt'] = word_dict_quiz_export['Num_Quiz_Attempt'].astype(int) + word_dict_quiz_export['Right Score'].notna()
word_dict_quiz_export['Num_Correct'] = word_dict_quiz_export['Num_Correct'].astype(int) + word_dict_quiz_export['Right Score'].fillna(0)
word_dict_quiz_export['Num_Wrong'] = word_dict_quiz_export['Num_Wrong'].astype(int) + word_dict_quiz_export['Wrong Score'].fillna(0)
word_dict_quiz_export['Last_Quiz'] = word_dict_quiz_export['Last_Quiz'].fillna('2001-01-01')
word_dict_quiz_export['Last_Quiz'] = np.where(word_dict_quiz_export['Last_Quiz'] < word_dict_quiz_export['Last Quiz'], word_dict_quiz_export['Last Quiz'], word_dict_quiz_export['Last_Quiz'])

num_attempts_post_update = word_dict_quiz_export['Num_Quiz_Attempt'].fillna(0).astype(int).sum()
num_correct_post_update = word_dict_quiz_export['Num_Correct'].fillna(0).astype(int).sum()
num_wrong_post_update = word_dict_quiz_export['Num_Wrong'].fillna(0).astype(int).sum()

attempt_count_condition = (num_attempts + len(quiz_result_df) != num_attempts_post_update)
correct_wrong_total_condition = (num_correct_post_update + num_wrong_post_update != num_attempts_post_update)

if (attempt_count_condition) or (correct_wrong_total_condition):
    raise Exception("Error in updating the quiz results.  Please check the code.")

word_dict_quiz_export = word_dict_quiz_export.drop(columns=['Right Score', 'Wrong Score', 'Last Quiz'])



## Manual Code

In [ ]:
quiz_result_export = quiz_result.copy()
quiz_result_export['Right Score'] = np.where((quiz_result_export['Meaning Correct']=="yes")&(quiz_result_export['Pinyin Correct']=="yes"), 1, 0)
quiz_result_export['Wrong Score'] = np.where(quiz_result_export['Right Score']==0, 1, 0)
quiz_result_export['Last Quiz'] = datetime.now().strftime('%Y-%m-%d')

quiz_result_export[['Right Score', 'Wrong Score', 'Last Quiz']].reset_index()

In [ ]:
word_dict_quiz_export = word_dict.merge(quiz_result_export[['Right Score', 'Wrong Score', 'Last Quiz']].reset_index(), on='Word Id', how='left')

num_attempts = word_dict_quiz_export['Num_Quiz_Attempt'].fillna(0).astype(int).sum()
word_dict_quiz_export['Num_Quiz_Attempt'] = word_dict_quiz_export['Num_Quiz_Attempt'].astype(int) + word_dict_quiz_export['Right Score'].notna()
word_dict_quiz_export['Num_Correct'] = word_dict_quiz_export['Num_Correct'].astype(int) + word_dict_quiz_export['Right Score'].fillna(0)
word_dict_quiz_export['Num_Wrong'] = word_dict_quiz_export['Num_Wrong'].astype(int) + word_dict_quiz_export['Wrong Score'].fillna(0)
word_dict_quiz_export['Last_Quiz'] = word_dict_quiz_export['Last_Quiz'].fillna('2001-01-01')
word_dict_quiz_export['Last_Quiz'] = np.where(word_dict_quiz_export['Last_Quiz'] < word_dict_quiz_export['Last Quiz'], word_dict_quiz_export['Last Quiz'], word_dict_quiz_export['Last_Quiz'])

num_attempts_post_update = word_dict_quiz_export['Num_Quiz_Attempt'].fillna(0).astype(int).sum()
num_correct_post_update = word_dict_quiz_export['Num_Correct'].fillna(0).astype(int).sum()
num_wrong_post_update = word_dict_quiz_export['Num_Wrong'].fillna(0).astype(int).sum()

attempt_count_condition = (num_attempts + len(quiz_result_export) != num_attempts_post_update)
correct_wrong_total_condition = (num_correct_post_update + num_wrong_post_update != num_attempts_post_update)

if (attempt_count_condition) or (correct_wrong_total_condition):
    raise Exception("Error in updating the quiz results.  Please check the code.")

word_dict_quiz_export = word_dict_quiz_export.drop(columns=['Right Score', 'Wrong Score'])